In [1]:
import numpy as np
import time
import cv2
import os
import pandas as pd
from os.path import isfile, join
import glob
from matplotlib import pyplot as plt


In [2]:
subset = pd.read_csv('DATA_csv/YOLO_information.csv')
del subset['Unnamed: 0']
subset['frame_id'] = subset['frame_id'].astype(int)
subset['id']='NOT IDENTIFIED'
iterate = subset.index

In [4]:
r=0
k=0
bf = cv2.BFMatcher()
output = []
sift = cv2.xfeatures2d.SIFT_create()
#sift=cv2.SIFT()
for i in iterate :
    crop1 = cv2.imread("CROPS/"+str(i)+".jpg")
    kp_1, desc_1 = sift.detectAndCompute(crop1, None)
    r+=1
    for j in iterate[r:]:
        crop2 = cv2.imread("CROPS/"+str(j)+".jpg")
        kp_2, desc_2 = sift.detectAndCompute(crop2, None)
        matches = bf.knnMatch(desc_1,desc_2,k=2)
        number_keypoints = 0
        good = []
        for m,n in matches:
            if m.distance < 0.75*n.distance:
                good.append([m])
        if len(kp_1) <= len(kp_2):
            number_keypoints = len(kp_1)
        else:
            number_keypoints = len(kp_2)
            
            #print("GOOD Matches:", len(good))
            #print("How good it's the match: ", len(good) / number_keypoints * 100, "%")
            
        pourcentage = len(good)/number_keypoints*100
        tmp = []
        
        if pourcentage >60:
#             img3 = cv2.drawMatchesKnn(crop1,kp_1,crop2,kp_2,good,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
#             plt.imshow(img3),plt.show()
#             img3=cv2.drawKeypoints(crop1,kp_1,img3)
#             plt.imshow(img3),plt.show()
            print("the person in  crop %d"%i + " is the same person in crop %d"%j)
            tmp.append(i)
            tmp.append(j)
            output.append(tmp)


                

the person in  crop 0 is the same person in crop 3
the person in  crop 0 is the same person in crop 6
the person in  crop 0 is the same person in crop 9
the person in  crop 0 is the same person in crop 12
the person in  crop 0 is the same person in crop 15
the person in  crop 0 is the same person in crop 18
the person in  crop 0 is the same person in crop 21
the person in  crop 0 is the same person in crop 24
the person in  crop 0 is the same person in crop 27
the person in  crop 0 is the same person in crop 33
the person in  crop 1 is the same person in crop 4
the person in  crop 2 is the same person in crop 5
the person in  crop 3 is the same person in crop 6
the person in  crop 3 is the same person in crop 9
the person in  crop 3 is the same person in crop 12
the person in  crop 3 is the same person in crop 15
the person in  crop 3 is the same person in crop 18
the person in  crop 3 is the same person in crop 21
the person in  crop 3 is the same person in crop 24
the person in  crop

In [5]:
ids = []
for index,element in enumerate(output):
    unique = set()
    find = False
    
    if(index==0):
        unique.add(element[0])
        unique.add(element[1])
        ids.append(unique)

    else:
        for index_id,id in enumerate(ids):
            if (element[0] in id):
                ids[index_id].add(element[1])
                find = True
            if (element[1] in id):
                ids[index_id].add(element[0])
                find = True

    
        if(not find):
            unique.add(element[0])
            unique.add(element[1])
            ids.append(unique)

ids = [list(i) for i in ids]


In [6]:
subset["id"] = -1
ids = [list(i) for i in ids]
id = 0
for sublist in ids : 
    for element in sublist : 
        subset.loc[subset.index==element,['id']]=id
    id = id+1
    

In [7]:
i= 20

for index in range(0, len(subset)):
    if(subset['id'][index]==-1):
        subset['id'][index] = i
        i = i + 1


C:\Users\DELL\Anaconda33\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [8]:
subset.to_csv('DATA_csv/SIFT_information.csv')